In [15]:
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
from keras import layers
import keras
import numpy as np

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
#from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt

from numpy.random import seed
import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

def create_sequences(values, time_steps=128):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

data = pd.read_csv(f'data/SKAB/anomaly-free/anomaly-free.csv', sep = ';')
scaler = StandardScaler()


data.index = pd.to_datetime(data['datetime'])
data.drop('datetime', axis = 1, inplace=True)

#train = data[data['anomaly'] == 0]
train = data.copy()
train = train.values
scaler.fit(train)
train = scaler.transform(train)

#test = data[data['anomaly'] == 1]

#test.drop(['anomaly', 'changepoint'], axis = 1, inplace = True)
#test = test.values
#test = scaler.transform(test)
#time_steps = dataset_size - 1

X = create_sequences(train, time_steps=59)
#X_test = create_sequences(test, time_steps=time_steps)


model = keras.Sequential(
        [
            layers.Input(shape=(X.shape[1], X.shape[2])),#, X.shape[3])),

            #layers.Dropout(rate=0.2),
            layers.Conv1D(
                filters=32, kernel_size=7, padding="same", strides=1, activation="relu"
            ),

            # layers.Dropout(rate=0.2),
            layers.Conv1D(
                filters=16, kernel_size=7, padding="same", strides=1, activation="relu"
            ),

            layers.Conv1DTranspose(
                filters=16, kernel_size=7, padding="same", strides=1, activation="relu"
            ),

            layers.Conv1DTranspose(
                filters=32, kernel_size=7, padding="same", strides=1, activation="relu"
            ),
            layers.Conv1DTranspose(
                filters=1, kernel_size=7, padding="same"),

            #layers.Dense(X.shape[2])
        ]
        )


model.compile(optimizer='adam', loss='mse')

n_epochs = 100

hist =  model.fit(X, X,
        epochs = n_epochs, batch_size = 8)

model.save_weights('./checkpoints/my_checkpoint')


Epoch 1/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8954
Epoch 2/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8914
Epoch 3/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8912
Epoch 4/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8911
Epoch 5/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8911
Epoch 6/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8910
Epoch 7/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8910
Epoch 8/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8910
Epoch 9/100
1169/1169 [==============================] - 2s 2ms/step - loss: 0.8910
Epoch 10/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8910
Epoch 11/100
1169/1169 [==============================] - 3s 2ms/step - loss: 0.8910
Epoch 12/100
1169/1169 [==============================] - 3s 2ms/step - lo

In [12]:
hist.history['loss']

[0.8951976299285889,
 0.8915168046951294,
 0.8911705613136292,
 0.8911343216896057,
 0.8910890817642212,
 0.8910223245620728,
 0.8910802006721497,
 0.8909891843795776,
 0.890963077545166,
 0.8909710049629211,
 0.8910157680511475,
 0.8909487724304199,
 0.8909597396850586,
 0.8909501433372498,
 0.8909522294998169,
 0.8909600377082825,
 0.8909488916397095,
 0.8909587860107422,
 0.8909683227539062,
 0.8909579515457153,
 0.8909305930137634,
 0.890982449054718,
 0.8909287452697754,
 0.8909458518028259,
 0.8909313082695007,
 0.8909456729888916,
 0.8909279704093933,
 0.8909955620765686,
 0.8909226655960083,
 0.8909643292427063,
 0.8909264206886292,
 0.8909506797790527,
 0.8909254670143127,
 0.8909571170806885,
 0.8909326791763306,
 0.8909435272216797,
 0.890921950340271,
 0.8909274935722351,
 0.8909388780593872,
 0.8909239172935486,
 0.8909409642219543,
 0.8909159898757935,
 0.8909426331520081,
 0.8909215927124023,
 0.8909980654716492,
 0.8909176588058472,
 0.8909160494804382,
 0.8909260034561